In [26]:
# modules
from tqdm import  tqdm
from glob import glob
import pandas as pd
import numpy as np
import os
from os import listdir
from os.path import basename as bn, join, split as sp

import librosa
import parselmouth
from parselmouth.praat import call
from scipy.io.wavfile import write

import praat_formants_python.praat_formants_python as pfp


# DATASET Path and Constants

In [27]:
# Read paths
ROOT_TIMIT_DATA_PATH = "/home/jeevan/datasets/TIMIT Acoustic-Phonetic Continuous Speech Corpus (LDC93S1)/TIMIT"

# Write paths
ALL_EXP_FOLDER = "./exports/"
(lambda fp : os.mkdir(fp) if not os.path.exists(fp) else 0)(ALL_EXP_FOLDER) #make export folder

# Vowel info Export CSV filename
ALL_TIMIT_VOWELS_IMP_FILENAME = "a_all-timit_vowels.csv"
ALL_TIMIT_VOWELS_IMP_FILEPATH = join(ALL_EXP_FOLDER, ALL_TIMIT_VOWELS_IMP_FILENAME)

# Vowel subset Export CSV filename
SUBSET_TIMIT_VOWELS_IMP_FILENAME = "b_subset-timit_vowels_vowlimLIM.csv"
SUBSET_TIMIT_VOWELS_IMP_FILEPATH = join(ALL_EXP_FOLDER, SUBSET_TIMIT_VOWELS_IMP_FILENAME)

# Vowel subset Export CSV filename
TIMIT_VOWEL_FORMANT_ESTIMATION_EXP_FILENAME = "c_timit-vowels_formant_estimation.csv"
TIMIT_VOWEL_FORMANT_ESTIMATION_EXP_FILEPATH = join(ALL_EXP_FOLDER, TIMIT_VOWEL_FORMANT_ESTIMATION_EXP_FILENAME)


# TMP Audio Export folder
TEMP_AUDIO_EXP_FOLDER = "./audio_exports"
(lambda fp : os.mkdir(fp) if not os.path.exists(fp) else 0)(TEMP_AUDIO_EXP_FOLDER) #make export folder

# TIMIT SAMPLING RATE
TIMIT_AUDIO_FS = 16000

### Import SUBSET TIMIT Vowel Info dataframe

In [28]:
VOWEL_LIMIT = 2
SUBSET_TIMIT_VOWELS_DF = pd.read_csv(SUBSET_TIMIT_VOWELS_IMP_FILEPATH.replace("LIM", f"{VOWEL_LIMIT}"))
# SUBSET_TIMIT_VOWELS_DF.set_index("index", inplace=True)
ALL_TIMIT_VOWEL_LIST = pd.unique(SUBSET_TIMIT_VOWELS_DF["vowel_name"])
print(SUBSET_TIMIT_VOWELS_DF["vowel_name"].value_counts())
SUBSET_TIMIT_VOWELS_DF

iy      4
ae      4
uh      4
ey      4
ah      4
aw      4
ux      4
ax      4
ay      4
oy      4
eh      4
ix      4
ow      4
axr     4
ao      4
ih      4
uw      4
aa      4
er      4
ax-h    4
Name: vowel_name, dtype: int64


,index,audio_filepath,wav_file,person_id,sex,start_sample,end_sample,duration_sample,start_second,end_second,duration_second,vowel_name
0,0,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SA2.WAV,MTRT0,M,19040,20720,1680,1.190000,1.295000,0.105000,iy
1,1,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SA1.WAV,MMGG0,M,31768,33269,1501,1.985500,2.079312,0.093812,iy
2,2,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SI730.WAV,FRAM1,F,34066,35184,1118,2.129125,2.199000,0.069875,iy
3,3,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SI567.WAV,FDMY0,F,32279,33690,1411,2.017438,2.105625,0.088188,iy
4,4,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SA2.WAV,MKLS1,M,28699,31624,2925,1.793688,1.976500,0.182812,ae
...,...,...,...,...,...,...,...,...,...,...,...,...
75,75,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SX338.WAV,FECD0,F,17880,18920,1040,1.117500,1.182500,0.065000,uh
76,76,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SX306.WAV,MWVW0,M,2912,3444,532,0.182000,0.215250,0.033250,ax-h
77,77,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SX401.WAV,MJDA0,M,21000,21377,377,1.312500,1.336062,0.023563,ax-h
78,78,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SX56.WAV,FBCH0,F,16925,17605,680,1.057813,1.100313,0.042500,ax-h


## FUNCTION: Measure Pitch of audio chunk | PARSEL MOUTH

In [29]:
def measure_pitch(audio_path: str) -> float:
    f0min, f0max = [75, 600]
    
    sound = parselmouth.Sound(audio_path) # read the sound
    pitch = call(sound, "To Pitch", 0, f0min, f0max) #create a praat pitch object
    mean_pitch = call(pitch, "Get mean", 0, 0, "Hertz") # get mean pitch
    return mean_pitch

## FUNCTION: Measure formants of audio chunk | PARSEL MOUTH

In [30]:
def measure_formants_psm(audio_path: str, vowel_name: str, start_sec: float, end_sec: float):
    f0min, f0max = [75, 600]
    sound = parselmouth.Sound(audio_path) # read the sound
    # pitch = call(sound, "To Pitch (cc)", 0, f0min, 15, 'no', 0.03, 0.45, 0.01, 0.35, 0.14, f0max)
    pitch = call(sound, "To Pitch", 0.0001, f0min, f0max)
    mean_pitch = call(pitch, "Get mean", 0, 0, "Hertz") # get mean pitch
    
    audio_chunk, fs = librosa.load(audio_path, sr=None, offset=start_sec, duration=(end_sec - start_sec))
    tmp_audio_file = os.path.join(TEMP_AUDIO_EXP_FOLDER, f"{vowel_name}.wav")
    write(tmp_audio_file, fs, audio_chunk)
    sound_frm = parselmouth.Sound(tmp_audio_file)
    # sound_frm = sound_frm.extract_part(rom_time=start_sec, to_time=end_sec, window_shape=0, relative_width=1, preserve_times=False) # read the sound chunk
    pointProcess = call(sound_frm, "To PointProcess (periodic, cc)", f0min, f0max)
    formants = call(sound_frm, "To Formant (burg)", 0.0025, 5, 5000, 0.025, 50)
    numPoints = call(pointProcess, "Get number of points")

    f1_list = []
    f2_list = []
    f3_list = []
    f4_list = []
    
    # Measure formants only at glottal pulses
    for point in range(0, numPoints):
        point += 1
        t  = call(pointProcess, "Get time from index", point)
        f1 = call(formants, "Get value at time", 1, t, 'Hertz', 'Linear')
        f2 = call(formants, "Get value at time", 2, t, 'Hertz', 'Linear')
        f3 = call(formants, "Get value at time", 3, t, 'Hertz', 'Linear')
        f4 = call(formants, "Get value at time", 4, t, 'Hertz', 'Linear')
        f1_list.append(f1)
        f2_list.append(f2)
        f3_list.append(f3)
        f4_list.append(f4)
    
    f1_list = [f1 for f1 in f1_list if str(f1) != 'nan']
    f2_list = [f2 for f2 in f2_list if str(f2) != 'nan']
    f3_list = [f3 for f3 in f3_list if str(f3) != 'nan']
    f4_list = [f4 for f4 in f4_list if str(f4) != 'nan']
    
    # calculate mean formants across pulses
    f1_mean = np.mean(f1_list)
    f2_mean = np.mean(f2_list)
    f3_mean = np.mean(f3_list)
    f4_mean = np.mean(f4_list)
    
    # calculate median formants across pulses, this is what is used in all subsequent calcualtions
    # you can use mean if you want, just edit the code in the boxes below to replace median with mean
    f1_median = np.median(f1_list)
    f2_median = np.median(f2_list)
    f3_median = np.median(f3_list)
    f4_median = np.median(f4_list)
    
    return {
        "pitch_mean_praat_base": mean_pitch,

        "F1_mean_praat_base":f1_mean,
        "F2_mean_praat_base":f2_mean,
        "F3_mean_praat_base":f3_mean,
        "F4_mean_praat_base":f4_mean,

        "F1_median_praat_base": f1_median,
        "F2_median_praat_base": f2_median,
        "F3_median_praat_base": f3_median,
        "F4_median_praat_base": f4_median,
    }

## FUNCTION: Measure formants of audio chunk | PRAAT FORMANTS

In [31]:
def measure_formants_pfp(audio_path, start_sec, end_sec):
    formants = pfp.formants_at_interval(
        audio_path, start_sec, end_sec, maxformant=5500, winlen=0.025, preemph=50
    )

    pitch_mean = measure_pitch(audio_path)
    pitch_mean = np.round(pitch_mean, 2)
    
    formants_mean = formants.mean(axis=0)
    formants_mean = list(formants_mean)[1:]  # skip time
    formants_mean = np.round(formants_mean, 2)  # round

    formants_median = np.median(formants, axis=0)
    formants_median = list(formants_median)[1:]  # skip time
    formants_median = np.round(formants_median, 2) # round


    return {
        "pitch_mean_praat_base": pitch_mean,

        "F1_mean_praat_base": formants_mean[0],
        "F2_mean_praat_base": formants_mean[1],
        "F3_mean_praat_base": formants_mean[2],

        "F1_median_praat_base": formants_median[0],
        "F2_median_praat_base": formants_median[1],
        "F3_median_praat_base": formants_median[2],
    }


### FUNCTION TESTS

In [32]:
audf, start, end, v = SUBSET_TIMIT_VOWELS_DF.loc[np.random.randint(0, len(SUBSET_TIMIT_VOWELS_DF)), ["audio_filepath", "start_second", "end_second", "vowel_name"]]
print(audf, start, end)
print(measure_formants_pfp(audf, start, end))

/home/jeevan/datasets/TIMIT Acoustic-Phonetic Continuous Speech Corpus (LDC93S1)/TIMIT/TRAIN/DR4/MRSP0/SX196.WAV 0.4875 0.62375
{'pitch_mean_praat_base': 128.75, 'F1_mean_praat_base': 536.65, 'F2_mean_praat_base': 1311.08, 'F3_mean_praat_base': 2237.73, 'F1_median_praat_base': 565.5, 'F2_median_praat_base': 1282.45, 'F3_median_praat_base': 2210.12}


## FUNCTION: TIMIT Vowel PITCH, FORMANT Estimation

In [33]:
def estimate_vowel_formants(vowel_info):
    audio_file = vowel_info["audio_filepath"]
    start_sec = vowel_info["start_second"]
    end_sec = vowel_info["end_second"]
    vowel_name = vowel_info["vowel_name"]

    formant_estimates = measure_formants_pfp(audio_file, start_sec, end_sec)
    # formant_estimates = measure_formants_psm(audio_file, vowel_name, start_sec, end_sec)

    new_vowel_info = dict(vowel_info) | formant_estimates

    return new_vowel_info

vinfo = SUBSET_TIMIT_VOWELS_DF.loc[np.random.randint(0, len(SUBSET_TIMIT_VOWELS_DF))]
estimate_vowel_formants(vinfo)

{'index': 36,
 'audio_filepath': '/home/jeevan/datasets/TIMIT Acoustic-Phonetic Continuous Speech Corpus (LDC93S1)/TIMIT/TRAIN/DR3/MWGR0/SX166.WAV',
 'wav_file': 'SX166.WAV',
 'person_id': 'MWGR0',
 'sex': 'M',
 'start_sample': 6372,
 'end_sample': 7120,
 'duration_sample': 748,
 'start_second': 0.39825,
 'end_second': 0.445,
 'duration_second': 0.04675,
 'vowel_name': 'ix',
 'pitch_mean_praat_base': 98.96,
 'F1_mean_praat_base': 539.75,
 'F2_mean_praat_base': 1403.9,
 'F3_mean_praat_base': 2798.67,
 'F1_median_praat_base': 527.31,
 'F2_median_praat_base': 1440.98,
 'F3_median_praat_base': 2827.77}

### Create and export Vowel Formant Estimation Result dataframe

In [34]:
VOWELS_FORMANT_DF = pd.DataFrame([estimate_vowel_formants(v_i) for i, v_i in tqdm(SUBSET_TIMIT_VOWELS_DF[0:].iterrows())])

columns = ['index',  'person_id', 'sex', 'duration_second', 'vowel_name', 
           'pitch_mean_praat_base', 
           'F1_mean_praat_base', 'F2_mean_praat_base', 'F3_mean_praat_base', 
           'F1_median_praat_base', 'F2_median_praat_base', 'F3_median_praat_base']
           
VOWELS_FORMANT_DF.to_csv(TIMIT_VOWEL_FORMANT_ESTIMATION_EXP_FILEPATH, columns=columns, index=False)

VOWELS_FORMANT_DF = VOWELS_FORMANT_DF.loc[:, columns]
VOWELS_FORMANT_DF

80it [00:00, 173.51it/s]


,index,person_id,sex,duration_second,vowel_name,pitch_mean_praat_base,F1_mean_praat_base,F2_mean_praat_base,F3_mean_praat_base,F1_median_praat_base,F2_median_praat_base,F3_median_praat_base
0,0,MTRT0,M,0.105000,iy,97.40,456.21,1906.70,2309.29,454.47,1921.02,2346.71
1,1,MMGG0,M,0.093812,iy,122.09,561.03,2072.62,2771.38,314.55,2110.85,2724.62
2,2,FRAM1,F,0.069875,iy,203.89,477.89,2144.54,2429.88,472.10,2222.18,2299.66
3,3,FDMY0,F,0.088188,iy,201.71,455.39,2633.34,2889.85,470.68,2638.78,2851.65
4,4,MKLS1,M,0.182812,ae,119.38,551.35,1599.19,2245.83,577.93,1540.95,2315.69
...,...,...,...,...,...,...,...,...,...,...,...,...
75,75,FECD0,F,0.065000,uh,202.33,538.71,1340.44,3263.19,558.87,1337.64,3271.06
76,76,MWVW0,M,0.033250,ax-h,129.07,1229.49,1983.71,2709.70,1278.98,1772.28,2690.29
77,77,MJDA0,M,0.023563,ax-h,146.48,736.37,1402.85,2308.02,736.37,1402.85,2308.02
78,78,FBCH0,F,0.042500,ax-h,206.85,1022.93,2420.79,3496.52,456.65,2185.32,3361.32


### Create and export Vowel Formant Estimation Result dataframe: JSON

In [35]:
VOWELS_FORMANT_DF.to_json(
    TIMIT_VOWEL_FORMANT_ESTIMATION_EXP_FILEPATH.replace(".csv", ".json"), index=False, orient="table"
)